# Lesson 4: Building Agents with LangChain
## From Manual Agents to Framework-Based Implementation

**Course**: Development of Agentic AI Systems for Advertising Campaign Analysis using Langchain Framework

**Duration**: 60-90 minutes

---

## Learning Objectives

In this notebook, you will:
1. Transform the manual ReAct agent from Lesson 3 into a LangChain-based implementation
2. Create custom tools using LangChain's Tool abstraction
3. Build and run your first LangChain agent with GROQ
4. Compare agent behavior with different configurations
5. Design multi-agent architectures

---

## Quick Theory: Why Use a Framework?

In Lesson 3, we built agents manually using raw LLM API calls. While educational, this approach has limitations:

| Manual Approach (Lesson 3) | Framework Approach (LangChain) |
|---------------------------|--------------------------------|
| Custom JSON parsing | Built-in parsing and error handling |
| Manual tool dispatch | Automatic tool selection and execution |
| No memory abstraction | Ready-to-use memory components |
| Reinventing patterns | Battle-tested implementations |

**LangChain provides:**
- **Tool abstraction**: Define tools once, use everywhere
- **Agent executors**: Handle the ReAct loop automatically
- **Prompt templates**: Reusable, parameterized prompts
- **Memory systems**: Conversation history management
- **Multi-provider support**: Same code works with different LLMs

---

## Setup

In [1]:
# Install required packages (run once)
# !pip install langchain langchain-groq langchain-core python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.2/603.2 kB 15.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.4/640.4 kB 15.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [langchain]17 [langchain]core]


In [10]:
import os
import json

# LangChain imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain.agents import create_agent
from langchain_groq import ChatGroq

print("Imports successful!")

Imports successful!


### API Configuration

We'll use **GROQ** as our LLM provider for fast inference.

**Get your API key**: https://console.groq.com

**Available Models**:
| Model | Description | Input Price | Output Price |
|-------|-------------|-------------|---------------|
| Llama 3.3 70B | Versatile, high-performance | $0.59/1M | $0.79/1M |
| Llama 3.1 8B | Fast, efficient for simple tasks | $0.05/1M | $0.08/1M |

In [ ]:
GROQ_API_KEY = ""
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [12]:
# Initialize the LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=1000
)

# Quick test
response = llm.invoke("Say 'Hello, LangChain!' in one line.")
print(f"LLM Response: {response.content}")

LLM Response: Hello, LangChain!


---

## Part 1: From Manual to LangChain - Tool Implementation

### Lesson 3 Recap: Manual Tool Definition

In Lesson 3, we defined tools like this:
```python
TTVAM_TOOLS = [
    {
        "name": "get_campaign_performance",
        "description": "Retrieves performance data...",
        "parameters": {...}
    }
]
```

### LangChain Approach: Tool Objects

LangChain uses `Tool` objects that combine the function with its metadata.

In [13]:
# Mock TTVAM Campaign Database (same as Lesson 3)
MOCK_CAMPAIGNS = {
    "1234": {
        "name": "Spring Campaign 2024",
        "broadcaster": "MTV",
        "reach": 48.5,
        "frequency": 3.8,
        "impacts": 5200000,
        "target": "Adults 25-54",
        "period_start": "2024-03-01",
        "period_end": "2024-03-31"
    },
    "5678": {
        "name": "Summer Campaign 2024",
        "broadcaster": "Sanoma",
        "reach": 52.3,
        "frequency": 4.2,
        "impacts": 6100000,
        "target": "Adults 25-54",
        "period_start": "2024-06-01",
        "period_end": "2024-06-30"
    },
    "9012": {
        "name": "Autumn Campaign 2024",
        "broadcaster": "MTV",
        "reach": 45.2,
        "frequency": 3.5,
        "impacts": 4800000,
        "target": "Female 25-44",
        "period_start": "2024-09-01",
        "period_end": "2024-09-30"
    }
}

print(f"Loaded {len(MOCK_CAMPAIGNS)} campaigns")

Loaded 3 campaigns


### Example 1: Creating a LangChain Tool

Let's transform the `get_campaign_performance` function from Lesson 3 into a LangChain Tool.

In [14]:


@tool
def get_campaign_data(spotgate_code: str) -> str:
    """
    Retrieve campaign data by Spotgate code.
    
    Args:
        spotgate_code: The Spotgate identification code
        
    Returns:
        String with campaign data or error message
    """
    if spotgate_code not in MOCK_CAMPAIGNS:
        return f"Error: Campaign with Spotgate code '{spotgate_code}' not found."
    
    campaign = MOCK_CAMPAIGNS[spotgate_code]
    
    result = f"""Campaign Data for Spotgate {spotgate_code}:
Name: {campaign['name']}
Broadcaster: {campaign['broadcaster']}
Target: {campaign['target']}
Period: {campaign['period_start']} to {campaign['period_end']}
Reach: {campaign['reach']}%
Frequency: {campaign['frequency']}
Total Impacts: {campaign['impacts']:,}"""
    
    return result

# Test the tool
print(get_campaign_data.invoke("1234"))

Campaign Data for Spotgate 1234:
Name: Spring Campaign 2024
Broadcaster: MTV
Target: Adults 25-54
Period: 2024-03-01 to 2024-03-31
Reach: 48.5%
Frequency: 3.8
Total Impacts: 5,200,000


In [15]:
# The @tool decorator already created a LangChain Tool
# Let's inspect it
print(f"Tool created: {get_campaign_data.name}")
print(f"Description: {get_campaign_data.description}")

Tool created: get_campaign_data
Description: Retrieve campaign data by Spotgate code.

Args:
    spotgate_code: The Spotgate identification code

Returns:
    String with campaign data or error message


### Example 2: More Tools for Campaign Analysis

In [16]:
@tool
def compare_campaigns(campaign_codes: str) -> str:
    """
    Compare performance metrics of two campaigns.
    
    Args:
        campaign_codes: Two comma-separated Spotgate codes (e.g., '1234,5678')
        
    Returns:
        String with comparison of reach, frequency, and impacts
    """
    codes = [code.strip() for code in campaign_codes.split(",")]
    
    if len(codes) != 2:
        return f"Error: Please provide exactly 2 campaign codes. You provided {len(codes)}."
    
    missing_codes = [code for code in codes if code not in MOCK_CAMPAIGNS]
    if missing_codes:
        return f"Error: Campaigns not found: {', '.join(missing_codes)}"
    
    camp1 = MOCK_CAMPAIGNS[codes[0]]
    camp2 = MOCK_CAMPAIGNS[codes[1]]
    
    result = f"""Campaign Comparison: {codes[0]} vs {codes[1]}

Campaign 1: {camp1['name']}
- Reach: {camp1['reach']}%
- Frequency: {camp1['frequency']}
- Impacts: {camp1['impacts']:,}

Campaign 2: {camp2['name']}
- Reach: {camp2['reach']}%
- Frequency: {camp2['frequency']}
- Impacts: {camp2['impacts']:,}

Analysis:
- Reach difference: {abs(camp1['reach'] - camp2['reach']):.1f}% ({'Campaign 1 higher' if camp1['reach'] > camp2['reach'] else 'Campaign 2 higher'})
- {'Campaign 1' if camp1['reach'] > camp2['reach'] else 'Campaign 2'} achieved better reach.
- {'Campaign 1' if camp1['impacts'] > camp2['impacts'] else 'Campaign 2'} generated more impacts."""
    
    return result


@tool
def calculate_kpi(params: str) -> str:
    """
    Calculate KPIs from campaign metrics.
    
    Args:
        params: Format 'spotgate_code,budget' (e.g., '1234,50000')
        
    Returns:
        String with calculated KPIs (CPM, cost per reach point)
    """
    try:
        parts = params.split(",")
        spotgate_code = parts[0].strip()
        budget = float(parts[1].strip())
    except (IndexError, ValueError):
        return "Error: Input should be 'spotgate_code,budget' (e.g., '1234,50000')"
    
    if spotgate_code not in MOCK_CAMPAIGNS:
        return f"Error: Campaign '{spotgate_code}' not found."
    
    campaign = MOCK_CAMPAIGNS[spotgate_code]
    
    # Calculate KPIs
    cpm = (budget / campaign['impacts']) * 1000
    cost_per_reach_point = budget / campaign['reach']
    
    result = f"""KPI Analysis for {campaign['name']}:

Input Data:
- Budget: €{budget:,.2f}
- Impacts: {campaign['impacts']:,}
- Reach: {campaign['reach']}%

Calculated KPIs:
- CPM (Cost per Mille): €{cpm:.2f}
- Cost per Reach Point: €{cost_per_reach_point:.2f}
- Efficiency: {'Good' if cpm < 10 else 'Average' if cpm < 15 else 'High cost'}"""
    
    return result

# Test
print(compare_campaigns.invoke("1234,5678"))

Campaign Comparison: 1234 vs 5678

Campaign 1: Spring Campaign 2024
- Reach: 48.5%
- Frequency: 3.8
- Impacts: 5,200,000

Campaign 2: Summer Campaign 2024
- Reach: 52.3%
- Frequency: 4.2
- Impacts: 6,100,000

Analysis:
- Reach difference: 3.8% (Campaign 2 higher)
- Campaign 2 achieved better reach.
- Campaign 2 generated more impacts.


In [17]:
# Create all tools list - these are already tool objects from the @tool decorator
tools = [
    get_campaign_data,
    compare_campaigns,
    calculate_kpi
]

print(f"Created {len(tools)} tools:")
for tool in tools:
    print(f"  - {tool.name}")

Created 3 tools:
  - get_campaign_data
  - compare_campaigns
  - calculate_kpi


---

## Part 2: Building the LangChain Agent

### Creating the Agent

LangChain's new API simplifies agent creation. Instead of managing prompts and executors separately,
we use `create_agent()` which handles the ReAct loop automatically.

In [18]:
# Create the agent with tools
system_prompt = """You are an expert advertising campaign analyst with access to the TTVAM database.

Answer questions about campaigns using the available tools. Think step by step:
1. Identify which tool(s) you need to use
2. Call the tools with the correct parameters
3. Analyze the results
4. Provide a clear answer to the user"""

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    debug=True  # Show reasoning steps
)

print("Agent ready!")

Agent ready!


### Running the Agent

Watch how the agent reasons through queries (debug=True shows the ReAct cycle):

In [22]:
# Query 1: Simple data retrieval
query1 = "What is the reach of campaign 1234?"
print(f"Query: {query1}")
print("="*60)

# Stream the agent response
for chunk in agent.stream({"messages": [{"role": "user", "content": query1}]}, stream_mode="updates"):
    print(chunk)

print("\n" + "="*60)

Query: What is the reach of campaign 1234?
[values] {'messages': [HumanMessage(content='What is the reach of campaign 1234?', additional_kwargs={}, response_metadata={}, id='db0bbb60-da6f-4ed4-b846-884ec6c255fe')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '8eng1rzfe', 'function': {'arguments': '{"spotgate_code":"1234"}', 'name': 'get_campaign_data'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 633, 'total_tokens': 651, 'completion_time': 0.036528581, 'completion_tokens_details': None, 'prompt_time': 0.033811693, 'prompt_tokens_details': None, 'queue_time': 0.050683454, 'total_time': 0.070340274}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_f8b414701e', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--04d6131b-c6ea-48c9-a003-2124eadadcc0-0', tool_calls=[{'name': 'get_campaign_data', 'arg

In [20]:
# Query 2: Comparison (requires tool selection)
query2 = "Compare campaigns 1234 and 5678. Which one performed better?"
print(f"Query: {query2}")
print("="*60)

for chunk in agent.stream({"messages": [{"role": "user", "content": query2}]}, stream_mode="updates"):
    print(chunk)

print("\n" + "="*60)

Query: Compare campaigns 1234 and 5678. Which one performed better?
[values] {'messages': [HumanMessage(content='Compare campaigns 1234 and 5678. Which one performed better?', additional_kwargs={}, response_metadata={}, id='c3548e07-d25e-4993-98f0-9da3ebc9d8f4')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '2rr2htsca', 'function': {'arguments': '{"campaign_codes":"1234,5678"}', 'name': 'compare_campaigns'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 638, 'total_tokens': 659, 'completion_time': 0.040032876, 'completion_tokens_details': None, 'prompt_time': 0.072630921, 'prompt_tokens_details': None, 'queue_time': 0.154772602, 'total_time': 0.112663797}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_43d97c5965', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--7d83b061-5df4-41cc-aee7-101bb86

In [21]:
# Query 3: Multi-step reasoning (multiple tool calls)
query3 = "Calculate the CPM for campaign 1234 with a budget of 50000 euros"
print(f"Query: {query3}")
print("="*60)

for chunk in agent.stream({"messages": [{"role": "user", "content": query3}]}, stream_mode="updates"):
    print(chunk)

print("\n" + "="*60)

Query: Calculate the CPM for campaign 1234 with a budget of 50000 euros
[values] {'messages': [HumanMessage(content='Calculate the CPM for campaign 1234 with a budget of 50000 euros', additional_kwargs={}, response_metadata={}, id='8d844187-c4d4-41c3-b946-51fbceccb99f')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'h8kb2qvfz', 'function': {'arguments': '{"params":"1234,50000"}', 'name': 'calculate_kpi'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 640, 'total_tokens': 660, 'completion_time': 0.028290999, 'completion_tokens_details': None, 'prompt_time': 0.033092004, 'prompt_tokens_details': None, 'queue_time': 0.05089829, 'total_time': 0.061383003}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_f8b414701e', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--9764fae0-f205-46b8-92e7-1a5ca5cdaee

---

## Part 3: Exercises

Now it's your turn! Complete the following exercises.

### Exercise 1: Create a New Tool

Create a tool called `list_campaigns` that lists all available campaigns with their basic info.

**Requirements:**
- The tool should take no input (or accept an empty string)
- Return a list of all campaigns with their names and Spotgate codes
- Include the target audience for each

In [ ]:
# Exercise 1: TODO - Implement the list_campaigns function
# Don't forget to add the @tool decorator!

@tool
def list_campaigns(query: str = "") -> str:
    """
    List all available campaigns in the database.
    
    Args:
        query: Not used, accepts any input
        
    Returns:
        String listing all campaigns with their codes and names
    """
    # TODO: Implement this function
    # Hint: Loop through MOCK_CAMPAIGNS and format the output
    pass

# Test your function
# print(list_campaigns())

In [ ]:
# Exercise 1 continued: Convert to LangChain Tool

# TODO: Add the @tool decorator to your list_campaigns function above
# The @tool decorator will automatically create a LangChain tool from your function
# No need to wrap it with Tool() - just add @tool before the function definition

### Exercise 2: Add Tool to Agent and Test

Add your new `list_campaigns` tool to the agent and test it with these queries:
1. "What campaigns are available?"
2. "Show me all campaigns targeting Adults 25-54"

In [ ]:
# Exercise 2: TODO - Create an enhanced agent with your new tool

# enhanced_tools = tools + [list_campaigns]

# TODO: Create a new agent with enhanced_tools
# enhanced_agent = create_agent(
#     model=llm,
#     tools=enhanced_tools,
#     system_prompt=system_prompt,
#     debug=True
# )

# TODO: Test with the queries above
# Example:
# for chunk in enhanced_agent.stream(
#     {"messages": [{"role": "user", "content": "What campaigns are available?"}]},
#     stream_mode="updates"
# ):
#     print(chunk)

### Exercise 3: Create a Report Generation Tool

Create a tool that generates a brief executive summary for a campaign.

**Requirements:**
- Input: Spotgate code
- Output: A formatted executive summary including:
  - Campaign name and period
  - Key metrics (reach, frequency, impacts)
  - A performance assessment (good/average/poor based on reach)

In [ ]:
# Exercise 3: TODO - Implement generate_report function

def generate_report(spotgate_code: str) -> str:
    """
    Generate an executive summary report for a campaign.
    
    Args:
        spotgate_code: The Spotgate identification code
        
    Returns:
        Formatted executive summary
    """
    # TODO: Implement this function
    # Hint:
    # 1. Check if campaign exists
    # 2. Retrieve campaign data
    # 3. Assess performance (reach > 50% = good, > 40% = average, else = needs improvement)
    # 4. Format as executive summary
    pass

# Test
# print(generate_report("1234"))

### Exercise 4: Multi-Agent Architecture Design

Design a multi-agent system for comprehensive campaign analysis. Fill in the architecture below.

In [ ]:
# Exercise 4: TODO - Design your multi-agent architecture

multi_agent_design = {
    "architecture_type": "",  # "sequential", "hierarchical", or "parallel"
    
    "agents": [
        # TODO: Define at least 3 specialized agents
        # Example:
        # {
        #     "name": "Data Agent",
        #     "role": "Retrieve campaign data",
        #     "tools": ["get_campaign_data", "list_campaigns"],
        #     "llm": "llama-3.1-8b (fast, simple tasks)"
        # }
    ],
    
    "workflow": [
        # TODO: Define the workflow steps
        # Example:
        # {"step": 1, "agent": "Data Agent", "action": "Fetch data"}
    ],
    
    "advantages": [
        # TODO: List at least 3 advantages of your design
    ],
    
    "challenges": [
        # TODO: List at least 2 potential challenges
    ]
}

print(json.dumps(multi_agent_design, indent=2))

### Exercise 5: Error Handling

Test your agent with edge cases and observe how it handles errors.

In [ ]:
# Exercise 5: Test edge cases

edge_cases = [
    "What is the reach of campaign 9999?",  # Non-existent campaign
    "Compare campaigns 1234, 5678, and 9012",  # More than 2 campaigns
    "What is the weather today?",  # Out of domain question
]

for query in edge_cases:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print("="*60)
    
    try:
        # Invoke the agent and get the final response
        result = agent.invoke({"messages": [{"role": "user", "content": query}]})
        # Extract the last message content
        final_message = result["messages"][-1].content
        print(f"Result: {final_message}")
    except Exception as e:
        print(f"Error: {e}")

# TODO: Document your observations below
# - How did the agent handle the non-existent campaign?
# - What happened with the 3-campaign comparison?
# - How did it respond to the out-of-domain question?

---
### Comparison: Manual vs LangChain

| Aspect | Manual (Lesson 3) | LangChain (Lesson 4) |
|--------|-------------------|----------------------|
| Lines of code | ~100 for agent loop | ~10 for agent setup |
| Error handling | Manual JSON parsing | Built-in |
| Tool dispatch | Custom logic | Automatic |
| Prompt format | Manual string building | Template-based |


---

## Resources

- **LangChain Documentation**: https://python.langchain.com/docs/introduction/
- **GROQ Console**: https://console.groq.com
- **ReAct Paper**: https://arxiv.org/abs/2210.03629
- **LangChain Agents Tutorial**: https://python.langchain.com/docs/tutorials/agents
